In [9]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import make_column_transformer


#read data
df = pd.read_csv('titanic_train.csv').set_index('PassengerId').drop(['Name', 'Ticket', 'Cabin'], axis=1).dropna()

#encode categorical data
sexes = ['female', 'male']
classes = [1, 2, 3]
ports = ['C', 'S', 'Q']
enc = preprocessing.OneHotEncoder(categories = [sexes, classes, ports], sparse=False)
df = pd.concat([df.drop(['Sex', 'Pclass', 'Embarked'], axis=1), pd.DataFrame(enc.fit_transform(df[['Sex', 'Pclass', 'Embarked']]), index=df.index)], axis=1)

#scale data
scaler = MinMaxScaler()
X = scaler.fit_transform(df.drop('Survived', axis=1).values.reshape(-1, 12))

y = df['Survived'].values
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

#train model
clf = SVC(kernel='linear', gamma=1, C=1000)
clf.fit(X, y)

# grid = GridSearchCV(clf, param_grid = grid_values, scoring = 'accuracy')
# grid.fit(X_train, y_train)
# y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test)

# print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
# print('Grid best parameter: ', grid.best_params_)
# print('Grid best score: ', grid.best_score_)
# print(grid.score(X_test, y_test))

#read test data
Xtest = pd.read_csv('titanic_test.csv').set_index('PassengerId').drop(['Name', 'Ticket', 'Cabin'], axis=1)

#preprocess test data
Xtest = pd.concat([Xtest.drop(['Sex', 'Pclass', 'Embarked'], axis=1),
                   pd.DataFrame(enc.fit_transform(Xtest[['Sex', 'Pclass', 'Embarked']]), index=Xtest.index)], axis=1)
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
atest = imp.fit_transform(Xtest.values.reshape(-1, 12))

#predictions
fields = ['PassengerId', 'Survived']
tstpred = clf.predict(atest)
rows = np.array(list(zip(Xtest.index, tstpred)))
#print(rows)
filename = 'ps.csv'
with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(rows)